In [13]:
def mymodel(act, units):
    text_in = Input(shape=(768,),name='text_input')#[768]
    img_in = Input(shape=(512,),name='image_input')#[512]
    
    
    avg_pool_text = Dense(1)(text_in)
    avg_pool_image = Dense(1)(img_in)
    
    concate = Concatenate(axis=1)([avg_pool_text, avg_pool_image])
    fc_1 = Dense(1, activation='relu')(concate)
    fc_2 = Dense(2, activation='sigmoid')(fc_1)
    fc_2 = Reshape((2, 1))(fc_2)
    
    
    concate_feature = Concatenate(axis=1)([text_in, img_in])#1280
    # concate_feature = Dropout(0.1)(concate_feature)
    concate_feature = Reshape((2, 640))(concate_feature)
    

    dot_result = Dot(axes=1)([fc_2, concate_feature])
    
    dot_result = Reshape((640,))(dot_result)
    
    # dot_result = Dense(512)(dot_result)
    # dot_result = Dense(256)(dot_result)
    # dot_result = Dense(128)(dot_result)
    dot_result = Dense(64)(dot_result)
    dot_result = Dense(32)(dot_result)
    
    
    output = Dense(units,activation=act)(dot_result)
    model = Model([text_in, img_in], output)
    return model

In [14]:
from keras import backend as K

def categorical_crossentropy_with_prior(y_true, y_pred, tau=1.0):
    """带先验分布的交叉熵
    注：y_pred不用加softmax
    """
    prior = [0.25,0.42,0.33]  # 自己定义好prior，shape为[num_classes]
    log_prior = K.constant(np.log([i + 1e-8 for i in prior]))
    for _ in range(K.ndim(y_pred) - 1):
        log_prior = K.expand_dims(log_prior, 0)
    y_pred = y_pred + tau * log_prior
    return K.categorical_crossentropy(y_true, y_pred, from_logits=True)

def sparse_categorical_crossentropy_with_prior(y_true, y_pred, tau=1.0):
    """带先验分布的稀疏交叉熵
    注：y_pred不用加softmax
    """
    prior = [0.21, 0.28, 0.43, 0.08]    # 自己定义好prior，shape为[num_classes][0.25,0.42,0.33]  [0.14,0.86] [0.21,0.79] [0.61,0.39][0.88,0.12]
    #[0.14, 0.48, 0.29, 0.08] [0.21, 0.28, 0.43, 0.08][0.61, 0.28, 0.09, 0.03]#!!!!!替换点
    log_prior = K.constant(np.log([i + 1e-8 for i in prior]))
    for _ in range(K.ndim(y_pred) - 1):
        log_prior = K.expand_dims(log_prior, 0)
    y_pred = y_pred + tau * log_prior
    return K.sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [15]:
from tensorflow.keras.layers import Input, Conv1D, Dropout, GlobalAveragePooling1D, Concatenate, Conv2D, GlobalAveragePooling2D, Dense, Embedding, LSTM, Dot, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.applications.densenet import DenseNet121
import tensorflow.keras.preprocessing.sequence as S
from tensorflow.keras.optimizers import Adam,SGD

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
import pandas as pd

import numpy as np

model = mymodel('softmax', 4)#sigmoid#!!!!!!!替换点
model.compile(loss=sparse_categorical_crossentropy_with_prior, optimizer=Adam(lr=1e-4), metrics=['sparse_categorical_accuracy'])

C:\Users\srika\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [16]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text_input (InputLayer)        [(None, 768)]        0           []                               
                                                                                                  
 image_input (InputLayer)       [(None, 512)]        0           []                               
                                                                                                  
 dense_7 (Dense)                (None, 1)            769         ['text_input[0][0]']             
                                                                                                  
 dense_8 (Dense)                (None, 1)            513         ['image_input[0][0]']            
                                                                                            

In [17]:
train_image_path = r"C:\Users\srika\Dropbox\PC\Downloads\fda jcomp\memotion3-SEFusion-main\features\train_image_feature.csv"
val_image_path = r"C:\Users\srika\Dropbox\PC\Downloads\fda jcomp\memotion3-SEFusion-main\features\val_image_feature.csv"
test_image_path = r"C:\Users\srika\Dropbox\PC\Downloads\fda jcomp\memotion3-SEFusion-main\features\test_image_feature.csv"

train_text_path = r"C:\Users\srika\Dropbox\PC\Downloads\fda jcomp\train_text_feature.csv"
val_text_path = r"C:\Users\srika\Dropbox\PC\Downloads\fda jcomp\memotion3-SEFusion-main\features\val_text_feature.csv"
test_text_path = r"C:\Users\srika\Dropbox\PC\Downloads\fda jcomp\test_text_feature.csv"

train_label_path = r"C:\Users\srika\Dropbox\PC\Downloads\fda jcomp\memotion3\memotion3\train.csv"
val_label_path = r"C:\Users\srika\Dropbox\PC\Downloads\fda jcomp\memotion3-val\val.csv"


def load_data(text_path, image_path, label_path):
    text_features = pd.read_csv(text_path)
    image_features = pd.read_csv(image_path)
    labels_all = pd.read_csv(label_path)['c2_labels2']#!!!!!!替换点
    
    # features = pd.concat([text_features,image_features],axis=1)
#     print(features.shape)
    
    return [text_features, image_features],labels_all

train_X, train_y = load_data(train_text_path, train_image_path, train_label_path)
val_X, val_y = load_data(val_text_path, val_image_path, val_label_path)

In [18]:
val_y

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0.036667,-0.068909,-0.033808,0.016712,-0.027774,-0.042045,0.021178,0.009470,-0.015205,0.030085,...,-0.031625,-0.020059,-0.047598,0.029185,0.010217,0.044470,-0.012868,-0.010856,-0.016980,0.025460
1,0.027375,-0.005918,-0.069443,0.026553,-0.021353,-0.034590,0.010487,0.023493,0.009369,0.043172,...,0.049398,-0.024270,-0.036418,0.026280,-0.023060,-0.032327,0.005382,-0.013913,-0.040872,0.028349
2,0.074567,-0.000144,-0.034651,0.083255,-0.022633,-0.015512,0.019777,0.036717,-0.054037,0.012454,...,0.012142,-0.021912,0.028434,-0.008833,-0.014144,0.015536,0.010804,-0.001046,-0.038776,0.026005
3,0.006274,-0.015384,-0.030182,0.012863,-0.026002,-0.005374,0.034981,0.002432,-0.011419,0.002794,...,0.107654,-0.096436,0.019344,0.005212,0.008244,-0.040635,0.019645,0.001776,-0.057708,-0.014324
4,-0.017345,-0.059889,-0.053226,-0.013396,-0.040297,-0.028825,-0.008367,0.040676,-0.006847,-0.063212,...,0.017247,-0.019986,0.012633,0.059567,0.029702,-0.036301,-0.058061,-0.008037,0.040331,0.016041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,0.068004,0.012145,-0.024851,-0.002959,0.010269,0.009514,-0.003389,0.060350,0.034644,-0.008912,...,-0.014312,0.014861,0.068546,0.028714,0.057049,-0.030759,0.002119,0.024616,-0.004120,0.005211
1496,-0.006596,-0.113643,-0.184955,-0.028554,0.031527,-0.045653,-0.031580,-0.028445,-0.023317,-0.022827,...,0.018726,-0.031922,-0.108153,-0.012376,-0.046006,0.021154,0.023139,-0.011649,-0.021621,-0.031844
1497,0.041475,-0.036922,-0.113450,0.014010,0.005299,-0.047104,0.027191,0.036364,0.021774,-0.003192,...,-0.055677,-0.063553,-0.013701,0.026461,0.013983,0.003503,0.015914,-0.022309,-0.066804,0.013954
1498,0.058579,-0.137619,-0.093569,0.055321,-0.026186,-0.069416,0.017880,0.053981,-0.023510,0.021551,...,0.000193,0.030102,-0.007603,0.018524,-0.020105,-0.006399,0.021237,-0.073116,0.024113,-0.007528


In [19]:
from keras.callbacks import LambdaCallback
from sklearn.metrics import f1_score, confusion_matrix
from keras.callbacks import ModelCheckpoint

filepath='best.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_sparse_categorical_accuracy', verbose=1, save_best_only=True,mode='max',period=1)

batch_print_callback = LambdaCallback(on_epoch_end=lambda batch,\
                    logs: print("train f1:", f1_score(train_y, \
                    model.predict(train_X).argmax(axis=-1), average='weighted'), \
                    "val f1:", f1_score(val_y, model.predict(val_X).argmax(axis=-1),\
                    average='weighted')))

model.fit(train_X,train_y,batch_size=256,epochs=30,
        validation_data=(val_X,val_y),
        callbacks=[batch_print_callback,checkpoint] 
)

Epoch 1/30


ValueError: in user code:

    File "C:\Users\srika\anaconda3\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\srika\anaconda3\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\srika\anaconda3\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\srika\anaconda3\lib\site-packages\keras\engine\training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\srika\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\srika\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 200, in assert_input_compatibility
        raise ValueError(f'Layer "{layer_name}" expe

In [20]:
from tensorflow.keras.models import load_model

model_load = load_model('bestmodel/taskC2_0.4700.h5',custom_objects={'sparse_categorical_crossentropy_with_prior': sparse_categorical_crossentropy_with_prior})

OSError: No file or directory found at bestmodel/taskC2_0.4700.h5

In [9]:
from sklearn.metrics import f1_score, confusion_matrix

In [12]:
# confusion_matrix(val_y, predict.argmax(axis=-1))

In [65]:
predict = model_load.predict(val_X)
f1_score(val_y, predict.argmax(axis=-1), average='weighted')

0.4706675292845772